In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import  plot_bar, plot_scatter
from core.s3 import S3AssetManager

In [2]:
notebook_name = "bios_segundo_experimento_adiflow"
s3 = S3AssetManager(notebook_name=notebook_name)

In [3]:
path_base = "raw/bios/Prueba Adiflow 2025-May-Jun.xlsx"
df = s3.read_excel(
    path_base,
    sheet_name='Datos'

)

In [4]:
df.head()

,Observación,Inclusión adiflow,Fecha pdn,OP,Opel,Tolva Origen,Tolva Destino,Línea zootécnica,Línea zootécnica extendida,Cod. Producto,...,Presión vapor (Prom. ponderado),% Alimentador pel (Prom. ponderado),Amperaje peletizado (Prom. ponderado),(kg/s Vapor x Ton),% Tiempo perdido,% Retornos,# retornos,kWh Pellet x Ton,% Agua Dos,kg Agua Dos
0,Típico,No comparables,2025-05-02,322185,48725,312,461,PORCICULTURA,PORCICULTURA,241700P539,...,6.134950,0.604793,320.757248,32982.156527,0.0,0.108335,2.000000,18.335190,0.009943,238.6
1,Outlier (Anómalo BD),No comparables,2025-05-02,322186,48716,321,464,PORCICULTURA,PORCICULTURA,234700P236,...,NaN,NaN,NaN,22425.959353,0.0,0.121214,1.000000,10.485870,0.009970,119.8
2,Típico,No comparables,2025-05-02,322186,48717,320,464,PORCICULTURA,PORCICULTURA,234700P236,...,14.777000,0.600000,286.846423,28577.972827,0.0,0.010415,0.666835,13.208803,0.009973,159.6
3,Típico,No comparables,2025-05-02,322187,48729,350,453,PORCICULTURA,PORCICULTURA,241540P034,...,33.468952,0.516905,114.216280,38399.297975,0.0,0.389220,2.671652,18.192579,0.009996,40.0
4,Típico,No comparables,2025-05-02,322180,48728,331,410,AVICULTURA,ENGORDE,114040,...,8.124000,0.529390,124.963872,27291.941193,0.0,0.037902,1.000000,24.559205,0.010023,201.2


# Recategorización

In [5]:
df["tiempo_perdido_cal"] = df[' % Tiempo perdido']*df['Tiempo procesado Pel (Hrs con paros)']
df["tiempo_sin_paros_cal"] = df['Tiempo procesado Pel (Hrs con paros)'] - df["tiempo_perdido_cal"]

df["tiempo_perdido_teorico_cal"] = df[' % Tiempo perdido']*df["Tiempo procesado Pel Teo (Hrs con paros)"]
df["tiempo_teorico_sin_paros_cal"] = df['Tiempo procesado Pel Teo (Hrs con paros)'] - df["tiempo_perdido_teorico_cal"]

df['rend_teorico_sin_paros'] = df["MP procesada Pel (Tons)"]/df["tiempo_teorico_sin_paros_cal"]


df.loc[:, "rend_cal"] = df["MP procesada Pel (Tons)"]/df["Tiempo procesado Pel (Hrs con paros)"]

df.loc[:, "rend_ton"] = df["MP procesada Pel (Tons)"]*df.loc[:, "rend_cal"]

df.loc[:, "eff_cal"] = df.loc[:, "rend_cal"] /df['Ton/h Pel (Teórica)']

df.loc[:, "rend_sin_paros_cal"] = df["MP procesada Pel (Tons)"]/df["tiempo_sin_paros_cal"]
df.loc[:, "eff_sin_paros_cal"] = df.loc[:, "rend_sin_paros_cal"] /df['rend_teorico_sin_paros']


bins = [0, 0.01, 0.02, np.inf]  # 0-1%, 1-2%, >2%
labels = [
    "0 - 1",
    "1 - 2",
    ">2"
]

df["% Agua Dos Segmento"] = pd.cut(
    df[" % Agua Dos"],
    bins=bins,
    labels=labels,
    right=True,  # incluye el límite superior
    include_lowest=True
)


bins = [0, 0.005, 0.01, 0.015, 0.02, 0.025]  # en escala 0 a 1
labels = [
    "0% - 0.5%",
    "0.5% - 1%",
    "1% - 1.5%",
    "1.5% - 2%",
    "2% - 2.5%"
]

df["% Agua Dos Segmento Fine"] = pd.cut(
    df[" % Agua Dos"],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)
df["dosificacion_agua"] = np.where(df[" % Agua Dos"]<=0.01, "0 - 1", ">1")


df["dosificacion_agua2"] = np.where(df[" % Agua Dos"]<=0.015, "0 - 1.5", ">1.5")


cond_batche = df[' # baches'] > 4

df["tipo_corrida"] = np.where(cond_batche, 'largo', 'corto')

cond_pel = df['Pellet'].isin([310,320])
df["Tipo Pel"] = np.where(cond_pel, 'grande', 'peque')


bins = [0, 0.05, 0.1, 0.15, 0.2, 0.25]  # en escala 0 a 1
labels = [
    "0% - 5%",
    "5% - 10%",
    "10% - 15%",
    "15% - 20%",
    ">20%"
]

df["pct_retornos"] = pd.cut(
    df[' % Retornos'],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)


### Motivos de Outlier


In [6]:
df.groupby(["Observación",  'Inclusión adiflow']).agg(
    dist_op=('OP', 'nunique'),
    avg_eff=(' % Eficiencia Pellet (sobre Teórico)', 'mean'),
    avg_rend=(' Ton/h Pel (ejecutada)', 'mean'),
    avg_vapor=(' (kg/s Vapor x Ton)', 'mean'),
    avg_peso=(' MP procesada (Tons)', 'mean'),
)

dist_op   avg_eff   avg_rend  \
Observación          Inclusión adiflow                                 
Outlier (Anómalo BD) Con Adiflow             47       NaN        NaN   
                     No comparables         101       NaN        NaN   
                     Sin Adiflow             55       NaN        NaN   
Outlier (Ton/h pel)  Con Adiflow             14  1.841342  30.689027   
                     No comparables           3  1.755008  29.250133   
                     Sin Adiflow              4  1.813666  30.227763   
Outlier (Vapor)      Con Adiflow             31  0.448563   7.476043   
                     No comparables          21  0.543193   9.053225   
                     Sin Adiflow             28  0.534913   8.915222   
Outlier (peso)       Con Adiflow              8  0.732006   5.873501   
                     No comparables          14  0.764236   7.490742   
                     Sin Adiflow             12  0.692341   6.821227   
Típico               Con Adiflow            609  0.828379   8.885273   
                     No comparables         914  0.838599   9.001684   
                     Sin Adiflow            528  0.847477   9.138658   

                                           avg_vapor   avg_peso  
Observación          Inclusión adiflow                           
Outlier (Anómalo BD) Con Adiflow        15741.155091  10.009367  
                     No comparables     17095.982563   9.205302  
                     Sin Adiflow        15502.164179   7.865574  
Outlier (Ton/h pel)  Con Adiflow        11160.238996  23.718702  
                     No comparables     12782.264614  30.817180  
                     Sin Adiflow         9983.010828  24.264910  
Outlier (Vapor)      Con Adiflow         -125.657852  17.595677  
                     No comparables       -67.322285  20.399927  
                     Sin Adiflow          -79.374732  18.020326  
Outlier (peso)       Con Adiflow        29415.405469  12.248269  
                     No comparables     19899.464148  14.170600  
                     Sin Adiflow        24384.290026  12.846322  
Típico               Con Adiflow        24522.260333  16.429041  
                     No comparables     26165.238516  17.261600  
                     Sin Adiflow        22651.145322  16.290873

# Condiciones

In [7]:
# TODO: CONDICIONES
cond_tipico = df['Observación'] == 'Típico'
cond_adiflow_inclusion = df['Inclusión adiflow'].isin(['Con Adiflow', 'Sin Adiflow'])

data_tot = df[cond_tipico & cond_adiflow_inclusion].copy()  # importante el .copy()
print("datos filtrados por tipico y solo con y sin adiflow", data_tot.shape)

#TODO: Lógica de negocio: si es Adiflow, entonces la dosis de agua > 0
cond_dosis_agua_cero = data_tot[" % Agua Dos"] == 0
cond_adiflow = data_tot["Inclusión adiflow"] == 'Con Adiflow'
data_invalid = data_tot[cond_dosis_agua_cero].copy()

print("Datos sin inclusión de agua:", data_invalid.shape)
data = data_tot.drop(data_invalid.index).reset_index(drop=True)

print(data.shape)
print(data_tot.shape)
print(data_invalid.shape, "Registros invalidos")


datos filtrados por tipico y solo con y sin adiflow (1261, 51)
Datos sin inclusión de agua: (349, 51)
(912, 51)
(1261, 51)
(349, 51) Registros invalidos


In [8]:
data_invalid_group = data_invalid.groupby(['Inclusión adiflow']).agg(
        inicio_prueba=("Fecha pdn", "min"),
        fin_prueba=("Fecha pdn", "max"),

        tot_ton_pel=('MP procesada Pel (Tons)', 'sum'),
        total_tiempo_perdido=("tiempo_perdido_cal", "sum"),
        sum_tiempo_sin_paro=('tiempo_sin_paros_cal', "sum"),
        sum_tiempo_con_paro=('Tiempo procesado Pel (Hrs con paros)', 'sum'),

        toneladas_teorica=(' MP procesada (Tons)', 'sum'),
        tiempo_teorico=('Tiempo procesado Pel Teo (Hrs con paros)', 'sum'),

        temp_acon_mean_mean =(' Temp ACO (Prom. ponderado)', "mean"),
        presin_vap_mean_mena=(" Presión vapor (Prom. ponderado)", "mean"),
        vapor_ton_mean =(" (kg/s Vapor x Ton)", "mean"),
        kwh_ton_mean =(" kWh Pellet x Ton", "mean"),
        pct_retornos_mean=(' % Retornos', 'mean'))

data_invalid_group["rend_total_con_paro"] = data_invalid_group["tot_ton_pel"]/data_invalid_group["sum_tiempo_con_paro"]
data_invalid_group["rend_total_sin_paro"] = data_invalid_group["tot_ton_pel"]/data_invalid_group["sum_tiempo_sin_paro"]
data_invalid_group["rend_teorico"] = data_invalid_group["toneladas_teorica"]/data_invalid_group["tiempo_teorico"]
data_invalid_group["eficiencia"] = data_invalid_group["rend_total_con_paro"]/data_invalid_group["rend_teorico"]*100
data_invalid_group

,inicio_prueba,fin_prueba,tot_ton_pel,total_tiempo_perdido,sum_tiempo_sin_paro,sum_tiempo_con_paro,toneladas_teorica,tiempo_teorico,temp_acon_mean_mean,presin_vap_mean_mena,vapor_ton_mean,kwh_ton_mean,pct_retornos_mean,rend_total_con_paro,rend_total_sin_paro,rend_teorico,eficiencia
Inclusión adiflow,,,,,,,,,,,,,,,,,
Con Adiflow,2025-05-14,2025-05-30,2682.01245,4.893073,421.612174,426.505247,2732.51845,323.182228,65.942305,22.929010,23164.925403,17.227625,0.179012,6.288346,6.361326,8.455039,74.373938
Sin Adiflow,2025-05-31,2025-06-14,2077.93123,38.930209,326.038501,364.968710,2116.36537,245.927777,56.347764,21.632772,18932.474256,18.005332,0.160727,5.693450,6.373269,8.605638,66.159540


In [9]:
data_invalid["Línea zootécnica"].value_counts()

PORCICULTURA    235
GANADERIA        50
EQUINOS          45
AVICULTURA       19
Name: Línea zootécnica, dtype: int64

## Rendimiento promedio filtrando registros con y sin adiflow, y los dosis de agua para adiflow>0


In [10]:
def group_by_sumary(df, cols=[]):


    data_group = df.groupby(['Inclusión adiflow']+cols).agg(
        inicio_prueba=("Fecha pdn", "min"),
        fin_prueba=("Fecha pdn", "max"),

        tot_ton_pel=('MP procesada Pel (Tons)', 'sum'),
        total_tiempo_perdido=("tiempo_perdido_cal", "sum"),
        sum_tiempo_sin_paro=('tiempo_sin_paros_cal', "sum"),
        sum_tiempo_con_paro=('Tiempo procesado Pel (Hrs con paros)', 'sum'),

        toneladas_teorica=(' MP procesada (Tons)', 'sum'),
        tiempo_teorico=('Tiempo procesado Pel Teo (Hrs con paros)', 'sum'),

        temp_acon_mean_mean =(' Temp ACO (Prom. ponderado)', "mean"),
        presin_vap_mean_mena=(" Presión vapor (Prom. ponderado)", "mean"),
        vapor_ton_mean =(" (kg/s Vapor x Ton)", "mean"),
        kwh_ton_mean =(" kWh Pellet x Ton", "mean"),
        pct_retornos_mean=(' % Retornos', 'mean')


    ).reset_index().sort_values(cols)


    data_group["rend_total_con_paro"] = data_group["tot_ton_pel"]/data_group["sum_tiempo_con_paro"]
    data_group["rend_total_sin_paro"] = data_group["tot_ton_pel"]/data_group["sum_tiempo_sin_paro"]
    data_group["rend_teorico"] = data_group["toneladas_teorica"]/data_group["tiempo_teorico"]
    data_group["eficiencia"] = data_group["rend_total_con_paro"]/data_group["rend_teorico"]*100
    return data_group


def group_by(df, cols=[]):


    data_group = df.groupby(['Inclusión adiflow']+cols, observed=False).agg(

        tot_ton_pel=('MP procesada Pel (Tons)', 'sum'),

        total_tiempo_perdido=("tiempo_perdido_cal", "sum"),
        sum_tiempo_sin_paro=('tiempo_sin_paros_cal', "sum"),
        sum_tiempo_con_paro=('Tiempo procesado Pel (Hrs con paros)', 'sum'),

        #toneladas_teorica=(' MP procesada (Tons)', 'sum'),
        toneladas_teorica=('MP procesada Pel (Tons)', 'sum'),
        tiempo_teorico=('Tiempo procesado Pel Teo (Hrs con paros)', 'sum'),
        #tiempo_teorico=('Tiempo procesado Pel (Hrs con paros)', 'sum'),

        temp_acon_mean_mean =(' Temp ACO (Prom. ponderado)', "mean"),
        presin_vap_mean_mena=(" Presión vapor (Prom. ponderado)", "mean"),
        vapor_ton_mean =(" (kg/s Vapor x Ton)", "mean"),
        kwh_ton_mean =(" kWh Pellet x Ton", "mean"),
        pct_retornos_mean=(' % Retornos', 'mean')

    ).reset_index().sort_values(cols)


    data_group["rend_total_con_paro"] = data_group["tot_ton_pel"]/data_group["sum_tiempo_con_paro"]
    data_group["rend_teorico"] = data_group["toneladas_teorica"]/data_group["tiempo_teorico"]

    data_group["rend_total_sin_paro"] = data_group["tot_ton_pel"]/data_group["sum_tiempo_sin_paro"]
    data_group["eficiencia"] = data_group["rend_total_con_paro"]/data_group["rend_teorico"]*100

    return data_group



def comparar_rendimiento_adiflow(by_total):
    """
    Calcula el cambio porcentual en el rendimiento total y promedio
    entre el uso de Adiflow y su ausencia.

    Parámetros:
    - by_total (pd.DataFrame): debe tener las columnas:
        - 'Inclusión adiflow'
        - 'rend_total_sin_paro'
        - 'rendimiento_promedio_sin_paro'

    Retorna:
    - dict con los cambios porcentuales
    """

    rend_con = by_total[by_total["Inclusión adiflow"] == "Con Adiflow"]["rend_total_con_paro"].values[0]
    rend_sin = by_total[by_total["Inclusión adiflow"] == "Sin Adiflow"]["rend_total_con_paro"].values[0]
    cambio_rend_tot_con_paro = ((rend_con - rend_sin) / rend_sin) * 100

    print(f"Cambio porcentual en rendimiento total con paros: {cambio_rend_tot_con_paro:.2f}%")

    rend_con = by_total[by_total["Inclusión adiflow"] == "Con Adiflow"]["rend_total_sin_paro"].values[0]
    rend_sin = by_total[by_total["Inclusión adiflow"] == "Sin Adiflow"]["rend_total_sin_paro"].values[0]
    cambio_rend_mean_sin_paro = ((rend_con - rend_sin) / rend_sin) * 100
    print(f"Cambio porcentual en rendimiento total sin paros: {cambio_rend_mean_sin_paro:.2f}%")

    rend_con = by_total[by_total["Inclusión adiflow"] == "Con Adiflow"]["eficiencia"].values[0]
    rend_sin = by_total[by_total["Inclusión adiflow"] != "Con Adiflow"]["eficiencia"].values[0]
    cambio_rend_total_con_paro = ((rend_con - rend_sin) / rend_sin) * 100
    print(f"Cambio porcentual en eficiencia total con paros: {cambio_rend_total_con_paro:.2f}%")

In [11]:
by_data = group_by_sumary(df=data, cols=[])
comparar_rendimiento_adiflow(by_data)

Cambio porcentual en rendimiento total con paros: 4.54%
Cambio porcentual en rendimiento total sin paros: -7.00%
Cambio porcentual en eficiencia total con paros: 4.79%


In [12]:
rename={
 'Inclusión adiflow': 'Inclusión adiflow',
 'inicio_prueba': "Inicio",
 'fin_prueba': "Fin",
 'tot_ton_pel': "Cantidad Peletizada (Ton)",
 'sum_tiempo_con_paro': "Tiempo Proceso (Horas)",
 "total_tiempo_perdido": "Tiempo en Paros (Horas)",
 'rend_total_con_paro': "Rendimiento (Ton/Hora)",
 'eficiencia': "Eficiencia (%)",
 'kwh_ton_mean': "Energía Promedio (KWhxTon)",
 'vapor_ton_mean': "Vapor Promedio (kg/s x Ton)",
 'temp_acon_mean_mean': "Temp Acon Promedio (°C)",
 'presin_vap_mean_mena': 'Presión Vapor Promedio',
 'pct_retornos_mean' : 'Retornos Promedio (%)'

}
by_data_beuty = by_data.rename(columns=rename)
by_data_beuty = by_data_beuty[rename.values()].round(2)
s3.save_dataframe(by_data_beuty, "sumary.csv")
by_data_beuty

,Inclusión adiflow,Inicio,Fin,Cantidad Peletizada (Ton),Tiempo Proceso (Horas),Tiempo en Paros (Horas),Rendimiento (Ton/Hora),Eficiencia (%),Energía Promedio (KWhxTon),Vapor Promedio (kg/s x Ton),Temp Acon Promedio (°C),Presión Vapor Promedio,Retornos Promedio (%)
0,Con Adiflow,2025-05-14,2025-05-30,8153.2,1102.55,40.19,7.39,79.78,16.27,25080.15,69.78,19.41,0.15
1,Sin Adiflow,2025-05-31,2025-06-14,7523.6,1063.58,151.91,7.07,76.13,16.33,23961.13,63.00,18.13,0.14


In [13]:
data_porc = data[data["Línea zootécnica"] == 'PORCICULTURA']
by_data_porc = group_by_sumary(df=data_porc, cols=[])
by_data_porc = by_data_porc.rename(columns=rename)
by_data_porc = by_data_porc[rename.values()].round(2)
s3.save_dataframe(by_data_porc, "sumary_porc.csv")
by_data_porc


,Inclusión adiflow,Inicio,Fin,Cantidad Peletizada (Ton),Tiempo Proceso (Horas),Tiempo en Paros (Horas),Rendimiento (Ton/Hora),Eficiencia (%),Energía Promedio (KWhxTon),Vapor Promedio (kg/s x Ton),Temp Acon Promedio (°C),Presión Vapor Promedio,Retornos Promedio (%)
0,Con Adiflow,2025-05-14,2025-05-30,6327.59,757.99,7.92,8.35,85.54,14.65,26219.16,73.06,22.43,0.15
1,Sin Adiflow,2025-05-31,2025-06-14,6325.73,842.54,150.94,7.51,76.21,15.14,24751.08,65.91,20.18,0.14


In [14]:

def calcular_cambio_porcentual(df, cols=[], table='No'):
    """
    df: resultado de group_by(cols)
    cols: columnas de agrupación adicionales que usaste
    """
    # Columnas de métricas (todas menos las de agrupación)
    metric_cols = [c for c in df.columns if c not in ['Inclusión adiflow'] + cols]

    # Pivotear el dataframe
    pivot_df = df.pivot_table(
        index=cols,
        columns='Inclusión adiflow',
        values=metric_cols,
        observed=False

    )

    # Aplanar columnas MultiIndex
    pivot_df.columns = ['{}_{}'.format(var, estado) for var, estado in pivot_df.columns]


    # Calcular cambios porcentuales para cada métrica
    for var in metric_cols:
        col_con = f'{var}_Con Adiflow'
        col_sin = f'{var}_Sin Adiflow'
        col_cambio = f'{var}_cambio_pct'

        pivot_df[col_cambio] = (
            (pivot_df[col_con] - pivot_df[col_sin]) / pivot_df[col_sin]
        ) * 100

    # Opcional: Redondear
    pivot_df = pivot_df.round(3)

    # Reset index para que quede como DataFrame
    pivot_df = pivot_df.reset_index()

    pivot_df = pivot_df.sort_values(by=cols, ascending=True)
    cols_ref = cols + [
        'tot_ton_pel_Con Adiflow',
        'eficiencia_Con Adiflow',
        'tot_ton_pel_Sin Adiflow',
        'eficiencia_Sin Adiflow',
        'eficiencia_cambio_pct',
        'rend_total_con_paro_Con Adiflow',
        'rend_total_con_paro_Sin Adiflow',
        'rend_total_con_paro_cambio_pct']

    pivot_df_dep = pivot_df[cols_ref].copy()
    pivot_df_dep = pivot_df_dep.rename(columns={'tot_ton_pel_Con Adiflow': "Toneladas Adiflow",
                                 'eficiencia_Con Adiflow': "Eficiencia Adiflow (%)",
                                 'eficiencia_cambio_pct': "Cambio Porcentual Eficiencia (%)",

                                 'tot_ton_pel_Sin Adiflow': "Toneladas Sin Adiflow",
                                 'eficiencia_Sin Adiflow': "Eficiencia Sin Adiflow (%)",

                                 'rend_total_con_paro_Con Adiflow': 'Rendimiento Adiflow (Ton/H)',
                                 'rend_total_con_paro_Sin Adiflow': 'Rendimiento Sin Adiflow (Ton/H)',
                                 'rend_total_con_paro_cambio_pct': 'Cambio Porcentual Rendimiento (%)'},
                        )
    if table == 'No':
        res = pivot_df
    else:
        res = pivot_df_dep


    return res

In [15]:
col = '% Agua Dos Segmento Fine'

data_dosis_agua = group_by(df=data_porc, cols=[col])
cambios_dosis_agua = calcular_cambio_porcentual(data_dosis_agua, cols=[col])
cambios_dosis_agua

,% Agua Dos Segmento Fine,eficiencia_Con Adiflow,eficiencia_Sin Adiflow,kwh_ton_mean_Con Adiflow,kwh_ton_mean_Sin Adiflow,pct_retornos_mean_Con Adiflow,pct_retornos_mean_Sin Adiflow,presin_vap_mean_mena_Con Adiflow,presin_vap_mean_mena_Sin Adiflow,rend_teorico_Con Adiflow,...,tiempo_teorico_cambio_pct,temp_acon_mean_mean_cambio_pct,presin_vap_mean_mena_cambio_pct,vapor_ton_mean_cambio_pct,kwh_ton_mean_cambio_pct,pct_retornos_mean_cambio_pct,rend_total_con_paro_cambio_pct,rend_teorico_cambio_pct,rend_total_sin_paro_cambio_pct,eficiencia_cambio_pct
0,0% - 0.5%,95.053,92.335,13.616,14.192,0.130,0.111,19.472,16.986,11.609,...,-29.574,21.913,14.638,15.895,-4.059,17.215,-0.613,-3.455,-3.209,2.944
1,0.5% - 1%,84.383,78.557,14.604,14.979,0.160,0.141,21.414,19.258,10.288,...,-9.757,12.054,11.197,5.753,-2.502,13.736,12.862,5.069,-2.651,7.417
2,1% - 1.5%,85.133,96.219,15.078,15.554,0.154,0.151,24.991,25.658,8.198,...,48.366,1.734,-2.602,0.807,-3.056,1.425,-9.376,2.426,-9.314,-11.522
3,1.5% - 2%,89.390,7.693,14.560,38.816,0.198,0.080,29.268,8.973,9.382,...,12.707,27.630,226.196,44.661,-62.490,147.667,1213.732,13.065,77.485,1061.929
4,2% - 2.5%,67.996,NaN,26.542,NaN,0.115,NaN,31.130,NaN,5.000,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data_dosis_agua

,Inclusión adiflow,% Agua Dos Segmento Fine,tot_ton_pel,total_tiempo_perdido,sum_tiempo_sin_paro,sum_tiempo_con_paro,toneladas_teorica,tiempo_teorico,temp_acon_mean_mean,presin_vap_mean_mena,vapor_ton_mean,kwh_ton_mean,pct_retornos_mean,rend_total_con_paro,rend_teorico,rend_total_sin_paro,eficiencia
0,Con Adiflow,0% - 0.5%,1053.95548,0.160644,95.349776,95.510420,1053.95548,90.785762,75.981786,19.472268,28098.274089,13.615539,0.129902,11.034979,11.609260,11.053571,95.053254
5,Sin Adiflow,0% - 0.5%,1550.10540,3.875690,135.734856,139.610546,1550.10540,128.909649,62.324350,16.985824,24244.649492,14.191576,0.110824,11.103068,12.024743,11.420098,92.335180
1,Con Adiflow,0.5% - 1%,3274.42542,7.540649,369.644713,377.185362,3274.42542,318.282124,73.961536,21.413999,26200.923780,14.604114,0.160016,8.681210,10.287808,8.858304,84.383477
6,Sin Adiflow,0.5% - 1%,3453.42864,69.451691,379.518928,448.970620,3453.42864,352.695855,66.005342,19.257689,24775.652236,14.978936,0.140690,7.691881,9.791520,9.099490,78.556556
2,Con Adiflow,1% - 1.5%,1922.24742,0.216720,275.204087,275.420808,1922.24742,234.473204,70.519318,24.990785,25605.409335,15.078446,0.153655,6.979311,8.198154,6.984807,85.132713
7,Sin Adiflow,1% - 1.5%,1264.92712,0.016670,164.229867,164.246538,1264.92712,158.036830,69.317576,25.658335,25400.481960,15.553709,0.151496,7.701393,8.004002,7.702175,96.219277
3,Con Adiflow,1.5% - 2%,72.97924,0.000000,8.702204,8.702204,72.97924,7.778940,74.054496,29.268129,26158.000102,14.560036,0.198155,8.386294,9.381643,8.386294,89.390462
8,Sin Adiflow,1.5% - 2%,57.26920,77.593192,12.120313,89.713506,57.26920,6.901914,58.022997,8.972548,18082.286226,38.816490,0.080008,0.638357,8.297583,4.725059,7.693283
4,Con Adiflow,2% - 2.5%,3.97950,0.000000,1.170510,1.170510,3.97950,0.795900,44.693329,31.129836,3413.832149,26.541588,0.115104,3.399799,5.000000,3.399799,67.995981
9,Sin Adiflow,2% - 2.5%,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
fig = plot_bar(df=data_dosis_agua.round(2),
         x_col="% Agua Dos Segmento Fine",
         y_col="eficiencia",
         group_col="Inclusión adiflow",
         order_x=['0% - 0.5%', '0.5% - 1%', '1% - 1.5%', '1.5% - 2%', '2% - 2.5%'],
         cat_base="Sin Adiflow",
         show_delta=True,
         hover_data_cols=["tot_ton_pel"])
fig.show()
s3.save_plotly_html(fig, "eficiencia_dosis_agua.html")

In [18]:
col_group = ['% Agua Dos Segmento Fine']
dos_agua_linea = group_by(df=data, cols=col_group)
cambios_dos_agua_linea = calcular_cambio_porcentual(dos_agua_linea, cols=col_group, table='Si')
s3.save_dataframe(cambios_dos_agua_linea.round(2),"cambios_dos_agua_linea.csv")
cambios_dos_agua_linea

,% Agua Dos Segmento Fine,Toneladas Adiflow,Eficiencia Adiflow (%),Toneladas Sin Adiflow,Eficiencia Sin Adiflow (%),Cambio Porcentual Eficiencia (%),Rendimiento Adiflow (Ton/H),Rendimiento Sin Adiflow (Ton/H),Cambio Porcentual Rendimiento (%)
0,0% - 0.5%,1056.950,94.059,1550.105,92.335,1.867,10.929,11.103,-1.568
1,0.5% - 1%,3909.407,83.971,3822.935,79.606,5.484,7.386,7.183,2.825
2,1% - 1.5%,2922.684,73.381,2011.643,85.626,-14.300,6.593,7.002,-5.847
3,1.5% - 2%,260.179,75.291,122.699,13.587,454.152,8.114,1.220,565.142
4,2% - 2.5%,3.980,67.996,16.220,83.161,-18.235,3.400,4.158,-18.235


In [19]:
col_group = ['Pellet']
group_by_pel = group_by(df=data, cols= ['Pellet'])
cambios_pel = calcular_cambio_porcentual(group_by_pel, cols= ['Pellet'], table='Si')
s3.save_dataframe(cambios_pel.round(2),"table_pelet.csv")
cambios_pel

,Pellet,Toneladas Adiflow,Eficiencia Adiflow (%),Toneladas Sin Adiflow,Eficiencia Sin Adiflow (%),Cambio Porcentual Eficiencia (%),Rendimiento Adiflow (Ton/H),Rendimiento Sin Adiflow (Ton/H),Cambio Porcentual Rendimiento (%)
0,310,2124.468,63.205,1985.344,58.049,8.883,10.534,9.675,8.883
1,320,3196.772,74.670,2937.848,70.391,6.079,12.445,11.732,6.079
2,330,786.486,72.816,748.692,65.068,11.908,3.641,3.253,11.908
3,340,853.736,100.598,683.676,86.979,15.658,5.030,4.349,15.658
4,350,1191.737,92.291,1168.042,105.883,-12.837,4.615,5.294,-12.837


In [20]:
col = 'Pellet'
data_porc["Pellet"] = "pel - "+ data_porc["Pellet"].astype(str)
data_dosis_agua = group_by(df=data_porc, cols=[col])
cambios_dosis_agua = calcular_cambio_porcentual(data_dosis_agua, cols=[col])
data_dosis_agua

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_89663/3293427351.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Inclusión adiflow,Pellet,tot_ton_pel,total_tiempo_perdido,sum_tiempo_sin_paro,sum_tiempo_con_paro,toneladas_teorica,tiempo_teorico,temp_acon_mean_mean,presin_vap_mean_mena,vapor_ton_mean,kwh_ton_mean,pct_retornos_mean,rend_total_con_paro,rend_teorico,rend_total_sin_paro,eficiencia
0,Con Adiflow,pel - 310,1184.66738,3.971551,87.720666,91.692217,1184.66738,71.080043,70.346340,5.845196,25371.361087,15.517460,0.123022,12.920043,16.666667,13.504998,77.520258
5,Sin Adiflow,pel - 310,1480.73636,42.655874,110.177955,152.833829,1480.73636,88.844182,35.178357,5.506914,24118.675706,17.115744,0.122891,9.688538,16.666667,13.439498,58.131228
1,Con Adiflow,pel - 320,3196.77178,2.672267,254.200109,256.872375,3196.77178,191.806307,77.790342,13.331533,23147.798779,13.684769,0.143255,12.444981,16.666667,12.575808,74.669885
6,Sin Adiflow,pel - 320,2937.84810,40.230149,210.186212,250.416361,2937.84810,176.270886,76.641924,9.894739,21275.449482,14.248561,0.130563,11.731854,16.666667,13.977359,70.391122
2,Con Adiflow,pel - 330,204.70912,0.016671,45.026515,45.043187,204.70912,40.941824,62.295313,8.124000,16705.660146,18.702575,0.137684,4.544730,5.000000,4.546413,90.894599
7,Sin Adiflow,pel - 330,270.91986,38.530008,61.296751,99.826759,270.91986,54.183972,60.253012,9.886633,23116.955142,16.728499,0.133960,2.713900,5.000000,4.419808,54.278004
3,Con Adiflow,pel - 340,549.70144,0.000000,106.123853,106.123853,549.70144,109.940288,58.754023,36.478161,21310.043735,16.741716,0.160279,5.179810,5.000000,5.179810,103.596208
8,Sin Adiflow,pel - 340,468.18360,29.521214,89.313301,118.834515,468.18360,93.636720,59.805164,30.357129,23470.768984,15.838393,0.138708,3.939795,5.000000,5.242037,78.795896
4,Con Adiflow,pel - 350,1191.73734,1.257524,257.000147,258.257671,1191.73734,238.347468,73.729285,42.259737,34889.002194,14.446144,0.187517,4.614528,5.000000,4.637108,92.290567
9,Sin Adiflow,pel - 350,1168.04244,0.000000,220.629746,220.629746,1168.04244,233.608488,74.759418,45.480331,31664.395246,14.545755,0.154648,5.294129,5.000000,5.294129,105.882589


In [21]:
fig = plot_bar(df=data_dosis_agua.round(2),
         x_col="Pellet",
         y_col="eficiencia",
         group_col="Inclusión adiflow",
         cat_base="Sin Adiflow",
         show_delta=True,
         hover_data_cols=["tot_ton_pel"])
fig.show()
s3.save_plotly_html(fig, "pel.html")

In [22]:
data["Dosis-Linea-Pel"] = data['% Agua Dos Segmento Fine'].astype(str)+ '-'+data['Línea zootécnica']+'-'+data["Pellet"].astype(str)
col_dieta_pel = "Dosis-Linea-Pel"

data_by_dieta_data_pel = group_by(df=data, cols=[col_dieta_pel])
cambios_by_dieta_data_ = calcular_cambio_porcentual(data_by_dieta_data_pel, cols=[col_dieta_pel])
cambios_by_dieta_data_ = cambios_by_dieta_data_.dropna()

fig = plot_bar(df=data_by_dieta_data_pel.round(2),
         x_col="Dosis-Linea-Pel",
         y_col="eficiencia",
         group_col="Inclusión adiflow",
         cat_base="Sin Adiflow",
         show_delta=True,
         hover_data_cols=["tot_ton_pel"])
fig.show()
s3.save_plotly_html(fig, "pel_dosis_linea.html")

In [23]:
data["Linea-Pel"] = data['Línea zootécnica']+'-'+data["Pellet"].astype(str)
col_dieta_pel = 'Linea-Pel'

data_by_dieta_data_pel = group_by(df=data, cols=[col_dieta_pel])

fig = plot_bar(df=data_by_dieta_data_pel.round(2),
         x_col="Linea-Pel",
         y_col="eficiencia",
         group_col="Inclusión adiflow",
         cat_base="Sin Adiflow",
         show_delta=True,
         hover_data_cols=["tot_ton_pel"])
fig.show()
s3.save_plotly_html(fig, "pel_linea.html")


In [24]:
col = 'dosificacion_agua'

data_dosis_agua = group_by(df=data, cols=[col])
fig = plot_bar(df=data_dosis_agua.round(2),
         x_col="dosificacion_agua",
         y_col="eficiencia",
         group_col="Inclusión adiflow",
         cat_base="Sin Adiflow",
         show_delta=True,
         hover_data_cols=["tot_ton_pel"])
fig.show()
s3.save_plotly_html(fig, "dosificacion_agua.html")


In [25]:

pel = "Tipo Pel"
data_by_pel_data = group_by(df=data, cols=[pel])
cambios_by_pel_data = calcular_cambio_porcentual(data_by_pel_data, cols=[pel])

fig = plot_bar(df=data_by_pel_data.round(2),
         x_col="Tipo Pel",
         y_col="eficiencia",
         group_col="Inclusión adiflow",
         cat_base="Sin Adiflow",
         show_delta=True,
         hover_data_cols=["tot_ton_pel"])
fig.show()
s3.save_plotly_html(fig, "tipo_pel.html")



In [26]:
fig = plot_scatter(
    df=data,
    x_col="rend_cal",
    y_col=' Presión vapor (Prom. ponderado)',
    group_col="Inclusión adiflow",
    hover_data_cols=["Tolva Origen"],
    title="Presión de Vapor vs Rendimiento",
    x_title="Rendimiento (Ton/H)",
)

fig.show()
s3.save_plotly_html(fig, "scatter_presion_vs_rendimiento.html")

In [27]:

fig = plot_scatter(
    df=data,
    x_col="rend_cal",
    y_col=' (kg/s Vapor x Ton)',
    group_col="Inclusión adiflow",
    hover_data_cols=["Tolva Origen"],
    title="Presión de Vapor vs Rendimiento",
    x_title="Rendimiento (Ton/H)",
)

fig.show()
s3.save_plotly_html(fig, "caudal_vs_rendimiento.html")

In [28]:
fig = plot_scatter(
    df=data,
    x_col="rend_cal",
    y_col=' Temp ACO (Prom. ponderado)',
    group_col="Inclusión adiflow",
    hover_data_cols=["Tolva Origen", 'MP procesada Pel (Tons)'],
    title="Temperatura Acondicionador vs Rendimiento",
    x_title="Rendimiento (Ton/H)",
)

fig.show()
s3.save_plotly_html(fig, "temp_vs_rendimiento.html")